In [1]:
import sys
import json
sys.path.append('..')
sys.path.append('../questions_construction')

from src.common import *
from model_performances import *
from copy import deepcopy

import pandas as pd

EVERYTHING_POSTFIX = 'everything'

In [14]:
def to_df(results_all, answer_type, plan_length, domain):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER : 'T/F',
        FREE_ANSWER : 'Free'
    }
    
    index = []
    data = []
    for question_category in QUESTION_CATEGORIES + [ALL_CATEGORIES_KEY]:
        for ramifications in RAMIFICATION_TYPES:
            index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
            data_columns = {}
            for model_name in PROMPT_MODEL_NAMES:
                for prompt_type in PROMPT_TYPES:
                    instance = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type)
                    if not len(instance) == 1: raise ValueError(f'len(instance) == {len(instance)}')
                    data_columns[(model_name, prompt_type)] = instance[0][SK_RESULT]
            data.append(data_columns)
    return pd.DataFrame(data, index = index)

def df_to_latex_table(df):
        latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
        return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')

def assemble_table(results_all, answer_type, domain, score_key=None):
    latex_table_all = ''
    with open('latex_table_template/top') as f:
        latex_table_all += f.read() + '\n'
    latex_table_all += '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length, domain)) for plan_length in PLAN_LENGTHS])
    with open('latex_table_template/bottom') as f:
        latex_table_all += f.read()
    
    caption = f'{answer_type}, {score_key} scores for {domain}'.replace('_', ' ')
    latex_table_all = latex_table_all.replace('REPLACE_CAPTION_KEY', caption)
    
    return latex_table_all


In [15]:
answer_type = TRUE_FALSE_ANSWER #FREE_ANSWER#
score_key = 'accuracy'
results_all = open_jsonl(os.path.join(STATISTICS_PATH, save_stats_file(answer_type, score_key)))

domain = ALL_DOMAINS_KEY
for domain in DOMAIN_NAMES + [ALL_DOMAINS_KEY]:
    latex_table_all = assemble_table(results_all, answer_type, domain, score_key=score_key)
    with open(os.path.join(STATISTICS_PATH, 'tables', f'{domain}.{answer_type}.{score_key}.tex'), 'w') as f:
        f.write(latex_table_all)


/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_22854/4158211881.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_22854/4158211881.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
/var/folders/_0/bqv6